# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [4]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import json

In [2]:
# any general notebook setup, like log formatting

In [5]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [6]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(from_coord, to_coord):
    '''
    @param from_coord: Tuple of latitud/longitude coordinates in degrees
    @param to_coord: Tuple of latitud/longitude coordinates in degrees
    
    @return Distance between two coordinates in Km
    '''
    # raise NotImplemented()

    # approximate radius of earth in km
    R = 6371.0

    lat1 = radians(from_coord[0])
    lon1 = radians(from_coord[1])
    lat2 = radians(to_coord[0])
    lon2 = radians(to_coord[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance



In [7]:
def test_calculate_distance():
    # raise NotImplemented()
    expected = 58.79508979190137
    actual = calculate_distance(NEW_YORK_BOX_COORDS[0], NEW_YORK_BOX_COORDS[1])
    assert actual == expected
test_calculate_distance()

In [8]:
def add_distance_column(dataframe):
#     raise NotImplemented()
    dataframe = dataframe.reset_index()
    distances = []
    for index, row in dataframe.iterrows():
        distances.append(calculate_distance((row['pickup_longitude'], row['pickup_latitude']), (row['dropoff_longitude'], row['dropoff_latitude'])))
    print(distances)
    dataframe['distance'] = distances    

### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [18]:
def find_taxi_csv_urls():
    #  raise NotImplemented()
    response = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    a_tags = soup.find_all(lambda tag:tag.name == 'a' and 'title' in tag.attrs and tag.attrs['title'] == "Yellow Taxi Trip Records")
    hrefs = [a['href'] for a in a_tags]
    hrefs_years=[]
    year = 2009
    for i in range(1, 7):
        hrefs_years.extend(list( filter(lambda href: str(year) in href, hrefs)))
        year += 1
    hrefs_2015 = []
    hrefs_2015.extend(list( filter(lambda href: '2015' in href, hrefs)))
    for i in range(6):
        hrefs_2015.pop()
    all_hrefs = hrefs_years + hrefs_2015 
    return all_hrefs

def get_and_clean_month_taxi_data(url):
    # raise NotImplemented()
    df = pd.read_csv(url)
    print(df.head())
    return df
    

def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    #for csv_url in all_csv_url:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
    print(all_csv_urls[0])
    dataframe = get_and_clean_month_taxi_data(all_csv_urls[0])
    add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
    all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

get_and_clean_taxi_data()

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-01.csv


OverflowError: signed integer is greater than maximum

### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [9]:
def load_and_clean_uber_data(csv_file):
#     raise NotImplemented()
    df = pd.read_csv(csv_file)
    df.drop(['passenger_count', 'key', 'fare_amount'], axis = 1, inplace = True)
    df = df.iloc[: , 1:]
    df.drop(df[df['pickup_longitude'] < -74.242330].index.intersection(df[df['pickup_longitude'] > -73.717047].index), inplace=True)
    df.drop(df[df['dropoff_longitude'] < -74.242330].index.intersection(df[df['dropoff_longitude'] > -73.717047].index), inplace=True)
    df.drop(df[df['pickup_latitude'] < 40.560445].index.intersection(df[df['pickup_latitude'] > 40.908524].index), inplace=True)
    df.drop(df[df['dropoff_latitude'] < 40.560445].index.intersection(df[df['dropoff_latitude'] > 40.908524].index), inplace=True)
    return df
    
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe
get_uber_data()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217
1,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325
2,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647
3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349
4,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247
...,...,...,...,...,...
199995,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297
199996,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620
199997,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588
199998,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415


### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [23]:
def clean_month_weather_data_hourly(csv_file):
#     raise NotImplemented()
    df = pd.read_csv(csv_file)
    df.drop(df.columns.difference(['DATE','HourlyPrecipitation', 'HourlyWindSpeed']), 1, inplace=True)
#     df['HourlyPrecipitation'] = df['HourlyPrecipitation'].fillna(-1)
#     df['HourlyPrecipitation'] = df['HourlyPrecipitation'].apply(lambda x: df.loc[df.HourlyPrecipitation != 'T', 'HourlyPrecipitation'].astype('float').mean())
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace('T', 0.00)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].str.replace(r's', '')
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].fillna(0.00)
    df['HourlyWindSpeed'] = df['HourlyWindSpeed'].fillna(df['HourlyWindSpeed'].astype(float).mean())
    return df
    
def clean_month_weather_data_daily(csv_file):
#     raise NotImplemented()
    df = pd.read_csv(csv_file)
    df.drop(df.columns.difference(['DATE','DailyAverageWindSpeed']), 1, inplace=True)
    df = df[df.DATE.str.contains("23:59:00")]
    df['DailyAverageWindSpeed'] = df['DailyAverageWindSpeed'].fillna(df['DailyAverageWindSpeed'].astype(float).mean())
    return df
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = \
        [
            "weather/weather_2009.csv",
            "weather/weather_2010.csv",
            "weather/weather_2011.csv",
            "weather/weather_2012.csv",
            "weather/weather_2013.csv",
            "weather/weather_2014.csv",
            "weather/weather_2015.csv"
        ]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data
load_and_clean_weather_data()

/Users/AWalter/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9,13,41,64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/AWalter/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (8,17,18,19,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/AWalter/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/AWalter/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (8,9,10,17,18,19,41,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, se

(                      DATE HourlyPrecipitation  HourlyWindSpeed
 0      2009-01-01T00:51:00                0.00        28.000000
 1      2009-01-01T01:00:00                 0.0        28.000000
 2      2009-01-01T01:51:00                0.00        29.000000
 3      2009-01-01T02:51:00                0.00        28.000000
 4      2009-01-01T03:51:00                0.00        22.000000
 ...                    ...                 ...              ...
 12034  2015-12-31T21:51:00                0.00        15.000000
 12035  2015-12-31T22:51:00                0.00         9.000000
 12036  2015-12-31T23:51:00                0.00         9.000000
 12037  2015-12-31T23:59:00                 0.0         8.881596
 12038  2015-12-31T23:59:00                 0.0         8.881596
 
 [86658 rows x 3 columns],
                       DATE  DailyAverageWindSpeed
 32     2009-01-01T23:59:00              18.500000
 71     2009-01-02T23:59:00              12.800000
 104    2009-01-03T23:59:00           

### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def dataframes_to_table():
    raise NotImplemented()

In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

dataframes_to_table(table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)